**This notebook creates De-Randomized Smoothed MalConv model for different ablation size and evaluate them.**

In [1]:
import torch
import numpy as np
from secml.array import CArray
from secml_malware.smoothed_malconv import get_dataset, create_smoothed_malconv, modify_dataset_for_smoothed_malconv, train_model, model_predict, get_majority_voting

In [38]:
#mal_path = "secml_malware/data/malware_samples/test_folder"
#ben_path = "secml_malware/data/goodware_samples/my_samples"
mal_path = "/Users/shoumik/Desktop/UMD 1st semester/Computer & Network Security/Project/Codes/MalConv-New/FullDataset/output/malware"
ben_path = "/Users/shoumik/Desktop/UMD 1st semester/Computer & Network Security/Project/Codes/MalConv-New/FullDataset/output/benign"
max_length = 2**20
num_ablations = 20
ablation_size = int(max_length/num_ablations)

In [39]:
X, y, file_names = get_dataset(mal_path, ben_path, max_length, 50)
print(len(X))
y = np.reshape(y, (-1, ))
new_X, new_y = modify_dataset_for_smoothed_malconv(X, y, num_ablations)
print(new_X.shape)
print(new_y.shape)
print(new_y)

100
(100, 20, 52428)
(100, 20)
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [40]:
tensor_X = torch.FloatTensor(new_X)
tensor_y = torch.FloatTensor(new_y)

In [41]:
smoothed_malconv_models = create_smoothed_malconv(num_ablations, ablation_size)
#print(smoothed_malconv_models)

In [42]:
smoothed_malconv_models = train_model(smoothed_malconv_models, num_ablations, tensor_X, tensor_y, epochs=5)

In [43]:
#print(smoothed_malconv_models[0]._model.predict(tensor_X[:, 0, :]))
y_preds_2d = model_predict(smoothed_malconv_models, num_ablations, tensor_X)
print(y_preds_2d)

[[1 1 1 ... 0 1 0]
 [1 1 1 ... 0 1 1]
 [1 1 1 ... 0 1 1]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [44]:
votes = get_majority_voting(y_preds_2d, new_y.shape[0])
print(votes)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [45]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y, votes))

0.52
